In [ ]:
import pandas as pd
import requests
from PIL import Image
import pytesseract
from io import BytesIO
import re

# Load datasets
train_data = pd.read_csv('/home/rguktrkvalley/Desktop/train.csv')
test_data = pd.read_csv('/home/rguktrkvalley/Desktop/test.csv')

# Function to download images
def download_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

# Function to perform OCR and extract text from images
def extract_text_from_image(image):
    try:
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(f"Error extracting text: {e}")
        return ""

# Function to clean and process extracted text
def clean_extracted_text(text):
    # Regex to find numbers followed by units (e.g., 12.5 gram, 500 ml, 2 cm)
    pattern = r'(\d+\.?\d*)\s*(gram|g|cm|centimeter|ml|ounce|kg|kilogram|litre|mm)'
    match = re.search(pattern, text.lower())  # Convert text to lowercase for matching
    if match:
        # Extract the number and unit
        number = match.group(1)
        unit = match.group(2)
        # Map shorthand to allowed unit (e.g., "g" -> "gram", "cm" -> "centimetre")
        unit_mapping = {
            'g': 'gram',
            'cm': 'centimetre',
            'ml': 'millilitre',
            'kg': 'kilogram',
            'mm': 'millimetre',
            'ounce': 'ounce',
            'litre': 'litre'
        }
        unit = unit_mapping.get(unit, unit)  # Default to the found unit if no mapping is available
        return f"{number} {unit}"
    else:
        return ""  # Return empty string if no valid entity found

# Main processing loop for the test dataset
predictions = []
for idx, row in test_data.iterrows():
    image_url = row['image_link']
    img = download_image(image_url)
    
    if img:
        extracted_text = extract_text_from_image(img)
        cleaned_text = clean_extracted_text(extracted_text)
        
        prediction = cleaned_text
        
        # Append the prediction to the list
        predictions.append({"index": row['index'], "prediction": prediction})
    else:
        predictions.append({"index": row['index'], "prediction": ""})

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)

# Save the predictions to CSV in the required format
output_file = '/home/rguktrkvalley/Desktop/test_out.csv'
predictions_df.to_csv(output_file, index=False)

print(f'Predictions saved to {output_file}')


In [7]:
import requests
from PIL import Image
from io import BytesIO

# Function to download and open the image
def download_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

# Example usage with an image URL
image_url = "/home/rguktrkvalley/Desktop/img.png"
img = download_image(image_url)

# Display the image if it was downloaded successfully
if img:
    img.show()  # This will open the image in the default viewer


Error downloading image: Invalid URL '/home/rguktrkvalley/Desktop/img.png': No scheme supplied. Perhaps you meant https:///home/rguktrkvalley/Desktop/img.png?
